# 총점 및 순위 계산

- 읽어들일 파일명 지정

In [26]:
# file_name 설정
전처리데이터셋 = '../data/구군/진구.csv'

In [31]:
import pandas as pd
df = pd.read_csv(f'{전처리데이터셋}',header=0, encoding='utf-8')
df.head()

,left,bottom,right,top,CCTV,경사도,공원,관공서,주차장,주택가,POP,gid
0,129.052,35.154,129.053,35.155,0,71,0,0,0,18,74.90,1
1,129.046,35.156,129.047,35.157,0,49,0,0,0,10,138.72,2
2,129.046,35.168,129.047,35.169,0,4,0,0,0,0,99.63,3
3,129.057,35.146,129.058,35.147,0,94,0,0,0,15,142.45,4
4,129.045,35.179,129.046,35.180,0,62,0,0,0,40,231.27,5


In [33]:
"""
군집1: 불연성쓰레기 배출량이 많은 군집
- 진구, 동래구, 남구, 북구, 해운대구, 사하구, 금정구, 사상구

군집2: 유동인구가 많은 군집
- 강서구, 기장군

군집3: 면적대비 주택수가 많은 군집
- 서구, 중구, 동구, 영도구, 연제구, 수영구
"""
score = ['POP','주택가','경사도','공원','주차장','관공서','CCTV']

cluster_1 = ['진구', '동래구', '남구', '북구', '해운대구', '사하구', '금정구', '사상구']
cluster_1_weight = [0.094, 0.088, 0.037, 0.019, 0.007, 0.043, 0.086]

cluster2 = ['강서구','기장군']
cluster_2_weight = [0.014, 0.037, 0.009, 0.008, 0.006, 0.024, 0.005]

cluster_3 = ['서구','중구','동구','영도구','연제구','수영구']
cluster_3_weight = [0.108, 0.162, 0.048, 0.017, 0.009, 0.034, 0.1]


In [34]:
df[score].describe()

,CCTV,경사도,공원,관공서,주차장,주택가,POP
count,3248.000000,3248.000000,3248.000000,3248.000000,3248.000000,3248.000000,1568.000000
mean,0.033251,20.036946,0.129002,0.007389,0.016933,6.012931,241.634713
std,0.186063,31.330134,2.343135,0.085655,0.162810,11.586068,258.964652
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.090000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,63.787500
50%,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,177.340000
75%,0.000000,33.000000,0.000000,0.000000,0.000000,6.000000,324.980000
max,3.000000,357.000000,65.000000,1.000000,4.000000,68.000000,2074.290000


In [36]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

norm = df.copy()
norm.fillna(0, inplace=True)

norm[score] = scaler.fit_transform(norm.loc[:, score])

reset = ['left','bottom','right','top','gid']

# 총점 계산

In [42]:
data = norm.copy()

for i in range(len(score)):
    cols_name = score[i]
    data[cols_name] = data[cols_name] * cluster_1_weight[i] # 군집 1
    # data[cols_name] = data[cols_name] * cluster_2_weight[i] # 군집 2
    # data[cols_name] = data[cols_name] * cluster_3_weight[i] # 군집 3
data_df = data.set_index(reset)

data_df['sum'] = data_df.sum(axis=1)
data_df.reset_index(inplace=True)
data_df.head()

,left,bottom,right,top,gid,CCTV,경사도,공원,관공서,주차장,주택가,POP,sum
0,129.052,35.154,129.053,35.155,1,0.0,0.198880,0.0,0.0,0.0,0.264706,0.036109,0.499694
1,129.046,35.156,129.047,35.157,2,0.0,0.137255,0.0,0.0,0.0,0.147059,0.066876,0.351190
2,129.046,35.168,129.047,35.169,3,0.0,0.011204,0.0,0.0,0.0,0.000000,0.048031,0.059235
3,129.057,35.146,129.058,35.147,4,0.0,0.263305,0.0,0.0,0.0,0.220588,0.068674,0.552568
4,129.045,35.179,129.046,35.180,5,0.0,0.173669,0.0,0.0,0.0,0.588235,0.111494,0.873398


# 등수 계산

In [49]:
data_df['rank'] = data_df['sum'].rank(method='dense', ascending=False).astype(int)

final = data_df.sort_values(by='rank')
final.reset_index(drop=True, inplace=True)

final.head(20)

,left,bottom,right,top,gid,CCTV,경사도,공원,관공서,주차장,주택가,POP,sum,rank
0,129.041,35.162,129.042,35.163,482,0.000000,0.812325,0.0,0.0,0.00,0.941176,0.327408,2.080910,1
1,129.042,35.162,129.043,35.163,957,0.000000,0.820728,0.0,0.0,0.00,0.852941,0.398527,2.072196,2
2,129.075,35.172,129.076,35.173,963,0.000000,0.277311,0.0,1.0,0.00,0.588235,0.136061,2.001607,3
3,129.029,35.149,129.030,35.150,2659,0.000000,0.198880,0.0,1.0,0.25,0.514706,0.000000,1.963585,4
4,129.060,35.145,129.061,35.146,874,0.000000,0.806723,0.0,0.0,0.00,0.691176,0.399616,1.897515,5
5,129.066,35.161,129.067,35.162,217,0.333333,0.098039,0.0,1.0,0.00,0.235294,0.086536,1.753202,6
6,129.051,35.143,129.052,35.144,625,0.000000,0.750700,0.0,0.0,0.00,0.823529,0.131047,1.705277,7
7,129.069,35.159,129.070,35.160,1215,0.333333,0.229692,0.0,0.0,0.00,0.941176,0.176952,1.681154,8
8,129.071,35.158,129.072,35.159,489,0.333333,0.308123,0.0,0.0,0.00,0.750000,0.283808,1.675265,9
9,129.071,35.151,129.072,35.152,487,0.333333,0.140056,0.0,0.0,0.00,0.970588,0.173009,1.616986,10


# 최종 결과 CSV 파일쓰기

In [50]:
final.to_csv('진구_결과_총점.csv',index=False, sep=',', encoding='utf-8')